In [1]:
import jieba
import re
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
news = pd.read_csv('F:\wiki20190720\data\sqlResult_1558435.csv',encoding='gb18030')

In [3]:
len(news)

89611

In [17]:
news_1 = news.dropna(subset=['source', 'content'])

In [15]:
def judgement(words):
    if words["source"]=='新华社'or words["source"]=='新华网' : #第7次作业时发现除了新华社以外有时候还会标记成新华网，都是新华社的新闻
            judge = 1
    else:judge = 0
    return pd.Series([judge, words['content']], index=['y', 'content'])

In [18]:
news_list = news_1.apply(judgement,axis=1)

In [19]:
news_list

,y,content
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...
5,0,受到A股被纳入MSCI指数的利好消息刺激，A股市场从周三开始再度上演龙马行情，周四上午金...
6,0,虽然至今夏普智能手机在市场上无法排得上号，已经完全没落，并于 2013 年退出中国市场，但是...
7,0,沙漠雄鹰：震荡有利消化套牢筹码\r\n 周四开盘上证50在银行券商大蓝筹带动下一度涨近...
8,0,（原标题：武汉警方一下子抓了808人，还都是俊男靓女！原来他们每天偷偷摸摸干这事！）\r\n...
9,0,6月21日，A股纳入MSCI指数尘埃落定，但当天被寄予厚望的券商股并未扛起反弹大旗。22...


In [20]:
corpus = news_list.content.tolist()

In [21]:
y_value = news_list.y.values.astype(np.int)

In [22]:
y_value

array([0, 0, 0, ..., 1, 1, 1])

In [23]:
words_cut = []
mask = []
for sentence in tqdm(corpus):
    if not isinstance(sentence, str):
        mask.append(False)
        continue
    mask.append(True)
    sentence = ''.join(re.findall(r'\w+', sentence))
    words_cut.append(' '.join(jieba.cut(sentence)))
len(words_cut)

  0%|                                                | 0/87052 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\wu\AppData\Local\Temp\jieba.cache
Loading model cost 1.738 seconds.
Prefix dict has been built succesfully.
100%|███████████████████████████████████| 87052/87052 [05:35<00:00, 259.29it/s]


87052

In [24]:
words_cut[0]

'此外 自 本周 6 月 12 日起 除 小米 手机 6 等 15 款 机型 外 其余 机型 已 暂停 更新 发布 含 开发 版 体验版 内测 稳定版 暂不受 影响 以 确保 工程师 可以 集中 全部 精力 进行 系统优化 工作 有人 猜测 这 也 是 将 精力 主要 用到 MIUI9 的 研发 之中 MIUI8 去年 5 月 发布 距今已有 一年 有余 也 是 时候 更新换代 了 当然 关于 MIUI9 的 确切 信息 我们 还是 等待 官方消息'

In [25]:
y_value = y_value[mask]

In [26]:
y_value

array([0, 0, 0, ..., 1, 1, 1])

##  使用tfidf进行文本向量化

In [27]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b",max_features=350) 
#这里pattern是使匹配长度从2变为了1，防止中文中一些一个字的词语被忽略
X = vectorizer.fit_transform(words_cut)

In [28]:
X.shape

(87052, 350)

In [29]:
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB

D:\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [30]:
y = y_value

In [31]:
X_train, X_test, Y_train, Y_test = train_test_split(X.toarray(), y, random_state=42, test_size=0.5)

## Bayes

In [33]:
x_train,x_test, y_train, y_test = train_test_split(X_train, Y_train, random_state=88, test_size=0.15)

In [34]:
Bayes = GaussianNB()
Bayes.fit(x_train, y_train)

GaussianNB(priors=None)

In [35]:
y_pred = Bayes.predict(x_test) 
y_pred_prob = Bayes.predict_proba(x_test)

In [36]:
Bayes.score(x_test,y_test)

0.9897380915913616

In [37]:
precision_score(y_test, y_pred)

0.9971103178650349

In [38]:
recall_score(y_test, y_pred)

0.9915483434753212

In [39]:
f1_score(y_test, y_pred)

0.9943215526739554

In [40]:
roc_auc_score(y_test, y_pred_prob[:, 1])

0.9871216332626317

## SVM

In [42]:
svm = SVC()
svm.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [43]:
svm.score(x_test,y_test)

0.9061111962015622

In [46]:
y_pred = svm.predict(x_test) 

In [47]:
precision_score(y_test, y_pred)

0.9061111962015622

In [48]:
recall_score(y_test, y_pred)

1.0

In [49]:
f1_score(y_test, y_pred)

0.9507432703897147

尝试调整参数class_weight为balanced

In [53]:
svm = SVC(class_weight = 'balanced')
svm.fit(x_train, y_train)

SVC(C=1.0, cache_size=200, class_weight='balanced', coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [54]:
svm.score(x_test,y_test)

0.9180578955429621

In [55]:
y_pred = svm.predict(x_test) 

In [56]:
precision_score(y_test, y_pred)

0.9998142299832807

In [57]:
recall_score(y_test, y_pred)

0.90973630831643

In [58]:
f1_score(y_test, y_pred)

0.9526506770510664

## Random forest

In [44]:
rfc = RandomForestClassifier(oob_score=True, class_weight='balanced',  random_state=88)
rfc.fit(x_train, y_train)

D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:453: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
D:\Anaconda3\lib\site-packages\sklearn\ensemble\forest.py:458: RuntimeWarning: invalid value encountered in true_divide
  predictions[k].sum(axis=1)[:, np.newaxis])


RandomForestClassifier(bootstrap=True, class_weight='balanced',
            criterion='gini', max_depth=None, max_features='auto',
            max_leaf_nodes=None, min_impurity_decrease=0.0,
            min_impurity_split=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=True, random_state=88,
            verbose=0, warm_start=False)

In [59]:
rfc.score(x_test,y_test)

0.9937203247051616

In [60]:
y_pred = rfc.predict(x_test) 

In [61]:
precision_score(y_test, y_pred)

0.9977969835621081

In [62]:
recall_score(y_test, y_pred)

0.9952670723461798

In [63]:
f1_score(y_test, y_pred)

0.9965304222729965

K-means

In [64]:
from sklearn.cluster import KMeans
from sklearn.metrics import calinski_harabaz_score  #用calinski_harabaz_score来估计，值越大越好

In [73]:
k_means = KMeans(n_clusters=2)
y_pred = k_means.fit_predict(x_train)

In [74]:
calinski_harabaz_score(x_train, y_pred)

3152.5666274956384

In [75]:
k_means = KMeans(n_clusters=4)
y_pred = k_means.fit_predict(x_train)

In [76]:
calinski_harabaz_score(x_train, y_pred)

2133.8850805455572

聚类变多时，效果下降

## 用Random forest找出可能抄袭新华社文章的报道

In [77]:
y_pred = rfc.predict(X) 

In [79]:
news_list.head()

,y,content
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...


In [80]:
news_list["y_predict"] = y_pred

In [81]:
news_list.head()

,y,content,y_predict
0,0,此外，自本周（6月12日）起，除小米手机6等15款机型外，其余机型已暂停更新发布（含开发版/...,0
1,0,骁龙835作为唯一通过Windows 10桌面平台认证的ARM处理器，高通强调，不会因为只考...,0
2,0,此前的一加3T搭载的是3400mAh电池，DashCharge快充规格为5V/4A。\r\n...,0
3,1,这是6月18日在葡萄牙中部大佩德罗冈地区拍摄的被森林大火烧毁的汽车。新华社记者张立云摄\r\n,1
4,0,（原标题：44岁女子跑深圳约会网友被拒，暴雨中裸身奔走……）\r\n@深圳交警微博称：昨日清...,0


In [82]:
news_copy_result = news_list.loc[(news_list.y == 0)& (news_list.y_predict) == 1]

In [83]:
news_copy_result.head(8)

,y,content,y_predict
78,0,央广网合肥6月19日消息（记者王利）6月19日上午6时，由公安部消防局组织的中部地区跨区域地...,1
595,0,由北京市旅游发展委员委、北京市科学技术委员会联合主办的2016年京津冀科普旅游活动启动仪...,1
689,0,为期9天的珠穆朗玛峰北坡“清洁行动”5月6日启动，清洁范围将在北坡大本营海拔5200米至...,1
722,0,点击图片进入下一页\r\n图为主炮对海射击。林健摄\r\n央广网北京6月23日消息 (林健 ...,1
941,0,（原标题：提醒在哥伦比亚中国公民注意安全）\r\n当地时间6月17日下午5时10分许，哥伦比...,1
1176,0,（原标题：山东龙口放火案罪犯徐孝福今日被执行死刑）\r\n@公正烟台消息，山东龙口放火案罪犯...,1
1187,0,7月1日，香港特别行政区将迎来成立和回归20周年。香港中华总商会会长蔡冠深在接受中国经济网记...,1
1559,0,中国证券网讯 财政部１３日发布消息称，经国务院批准，２０１７年财政部将在境外发行１４０亿...,1


# 比较优缺点:

### Linear Regression:

  优点:把数据拟合成线性关系,可以利用机器学习不断修正达到比较准确的线性函数值,好算。

缺点:如果给的数据非常复杂,不适用线性函数时,效果会非常差.

##  Logistic Regression

优点:计算快速,简便.需要调的参数很少,效果目前看来比较好.

缺点:无法得到哪些feature会影响结果.

# KNN

优点:很容易解释原理,方法也不复杂,感觉是分类的一种变形.

缺点:计算量大,耗时久,如果分组不分好的话很容易爆内存无法运行.

# Bayes

优点:有牢固的概率论体系支撑,也有固定的公式去计算.计算起来较快,对于一些不在考虑范围内的值也可以根据公式把它忽略,从而不会使结果受到较大影响.

缺点:如果是训练出来预测结果的话,如果各个条件相关程度很高,那么Bayes得到的结果就会不准,因为它的前提就是各个小事件之间是相互独立的,若相关性很强,则概率公式就不成立了.